In [1]:
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon

import folium
from folium import Choropleth, Marker
from folium.plugins import HeatMap, MarkerCluster

from learntools.core import binder
binder.bind(globals())
from learntools.geospatial.ex5 import *

ModuleNotFoundError: No module named 'learntools'

In [ ]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [ ]:
collisions = gpd.read_file("../input/geospatial-learn-course-data/NYPD_Motor_Vehicle_Collisions/NYPD_Motor_Vehicle_Collisions/NYPD_Motor_Vehicle_Collisions.shp")
collisions.head()
collisions.count()

In [ ]:
m_1 = folium.Map(location=[40.7, -74], zoom_start=11)

# Your code here: Visualize the collision data

#HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']], radius=15).add_to(m_1)
#for idx, row in collisions.iterrows():
#    Marker([row['LATITUDE'], row['LONGITUDE']]).add_to(m_1)

# Plot each polygon on the map
HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m_1)

#>>>GeoJson(two_mile_buffer.to_crs(epsg=4326)).add_to(m_1)

# Show the map
m_1
____

# Uncomment to see a hint
#q_1.hint()

# Show the map
embed_map(m_1, "q_1.html")

2

In [ ]:
hospitals = gpd.read_file("../input/geospatial-learn-course-data/nyu_2451_34494/nyu_2451_34494/nyu_2451_34494.shp")
hospitals.head()

In [ ]:
m_2 = folium.Map(location=[40.7, -74], zoom_start=11)

# Your code here: Visualize the hospital locations
#HeatMap(data=hospitals[['latitude', 'longitude']], radius=9).add_to(m_2)


#folium.Marker(location=[40.7, -74]).add_to(m_2)

#folium.Marker([53.210903, 6.598276], popup="My marker").add_to(m)

#Marker([row['LATITUDE'], row['LONGITUDE']]).add_to(m)

#..........sol
for idx, row in hospitals.iterrows():
    Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(m_2)

____

# Uncomment to see a hint
q_2.hint()

# Show the map
embed_map(m_2, "q_2.html")

In [ ]:
print(collisions.crs)
print(hospitals.crs)

In [ ]:
hospitals_area = hospitals.geometry.buffer(10000)
print("-")
union = hospitals_area.geometry.unary_union
print("-")
outside_range = collisions.loc[~collisions["geometry"].apply(lambda x: union.contains(x))]
print("-")
# Check your answer
q_3.check()
outside_range.head()

In [ ]:
percentage = round(100*len(outside_range)/len(collisions), 2)
print("Percentage of collisions more than 10 km away from the closest hospital: {}%".format(percentage))

4

In [ ]:
def best_hospital(collision_location):
    # Your code here
    idx_min = hospitals.geometry.distance(collision_location).idxmin()
    my_hospital = hospitals.iloc[idx_min]
    name = my_hospital["name"]

    # Select one release incident in particular
        #recent_release = releases.iloc[360]

# Measure distance from release to each station
#hospitals = collisions.geometry.distance(name.geometry)
#hospitals

    return name

#def best_hospital(collision_location):
#    idx_min = hospitals.geometry.distance(collision_location).idxmin()
#    my_hospital = hospitals.iloc[idx_min]
#    name = my_hospital["name"]
#    return name

# Test your function: this should suggest CALVARY HOSPITAL INC
print(best_hospital(outside_range.geometry.iloc[0]))

# Check your answer
q_4.check()

In [ ]:
#def under_highest_demand
#    idx_min = hospitals
#    h_d = hospitals
#    highest_demand = h_d["name"]

highest_demand = outside_range.geometry.apply(best_hospital).value_counts().idxmax()

In [ ]:
folium.Map(location=[40.7, -74], zoom_start=11)

6

In [ ]:
m_6 = folium.Map(location=[40.7, -74], zoom_start=11)

coverage = gpd.GeoDataFrame(geometry=hospitals.geometry).buffer(10000)
folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m_6)
HeatMap(data=outside_range[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m_6)
folium.LatLngPopup().add_to(m_6)

embed_map(m_6, 'm_6.html')

In [ ]:
lat_1 = 40.6740
long_1 = -73.7588

# Your answer here: proposed location of hospital 2
lat_2 = 40.6744
long_2 = -73.8682


# Do not modify the code below this line
try:
    new_df = pd.DataFrame(
        {'Latitude': [lat_1, lat_2],
         'Longitude': [long_1, long_2]})
    new_gdf = gpd.GeoDataFrame(new_df, geometry=gpd.points_from_xy(new_df.Longitude, new_df.Latitude))
    new_gdf.crs = {'init' :'epsg:4326'}
    new_gdf = new_gdf.to_crs(epsg=2263)
    # get new percentage
    new_coverage = gpd.GeoDataFrame(geometry=new_gdf.geometry).buffer(10000)
    new_my_union = new_coverage.geometry.unary_union
    new_outside_range = outside_range.loc[~outside_range["geometry"].apply(lambda x: new_my_union.contains(x))]
    new_percentage = round(100*len(new_outside_range)/len(collisions), 2)
    print("(NEW) Percentage of collisions more than 10 km away from the closest hospital: {}%".format(new_percentage))
    # Did you help the city to meet its goal?
    q_6.check()
    # make the map
    m = folium.Map(location=[40.7, -74], zoom_start=11)
    folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m)
    folium.GeoJson(new_coverage.geometry.to_crs(epsg=4326)).add_to(m)
    for idx, row in new_gdf.iterrows():
        Marker([row['Latitude'], row['Longitude']]).add_to(m)
    HeatMap(data=new_outside_range[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m)
    folium.LatLngPopup().add_to(m)
    display(embed_map(m, 'q_6.html'))
except:
    q_6.hint()